In [1]:
import numpy as np
import pandas as pd
from characters.sparkle import Sparkle
from characters.qingque import Qingque
from characters.silver_wolf import Silver_Wolf
from characters.fu_xuan import Fu_Xuan
from utils.arena import Arena
from utils.data_table import Action_Table

In [2]:
def Initialize_QQ_Mono_Quantum():
  arena = Arena()
  QQ = Qingque(arena = arena, Base_Atk = qq_batk, AtkPct = qq_atkp, Total_Atk = qq_tatk, CritRate = qq_cr, CritDmg = qq_cd, DmgPct = qq_dmgp, Spd = qq_Spd, behavior = qq_bh)
  SW = Silver_Wolf(arena = arena, Spd = sw_Spd)
  FX = Fu_Xuan(arena = arena, tech = fx_tech, Spd = fx_Spd, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = QQ, Spd = spk_Spd, ER = 1.194, tech = True)
  return arena, QQ, Spk, SW, FX

In [3]:
def QQ_Mono_Quantum(enlvl = 95, quantum_weak = True, verbose = False):
    action_df = Action_Table()

    arena, qq, spk, sw, fx = Initialize_QQ_Mono_Quantum()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < 5:
        characters = [qq, spk, sw, fx]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
            arena.action_advance(lowest_gauge)
            qq.tiles += 1
        #Determine the next character to act based on who has 0 action gauge
        if fx.action_gauge == 0:
        
        #Fu Xuan's actions
            if verbose:
                print("Fu Xuan takes a turn")
            if arena.sp > 0 and fx.skill_counter <= 0:
                action_df.append(arena, fx, "Skill")
                ##TODO: Add FX's 12% CR Buff:
                fx.skill(verbose = verbose)
            else:
                action_df.append(arena, fx, "Basic")
                fx.basic(verbose = verbose)
            fx.update_buffs()
            fx.action_reset()

        elif sw.action_gauge == 0:
        
        #Silver Wolf's actions
            if verbose:
                print("Silver Wolf takes a turn")
            if arena.sp > 0 and sw.skill_counter <= 0:
                action_df.append(arena, sw, "Skill")
                sw.skill(verbose = verbose)
            else:
                action_df.append(arena, sw, "Basic")

                sw.basic(verbose = verbose)
            sw.update_buffs()
            sw.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0:
                action_df.append(arena, spk, "Skill")
                spk.skill(verbose = verbose)
            else:
                action_df.append(arena, spk, "Basic")
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.append(arena, spk, "Ultimate")
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()

        elif qq.action_gauge == 0:
        
        #QQ's Actions
            qq.ult_expected_damage = 0
            qq.ult_max_damage = 0
            if qq.tile_battle == True:
                draws = arena.sp + 1
                qq.tile_battle = False
            else:
                draws = arena.sp
            qq.skill(verbose)
            if qq.draws <= arena.sp:
                qq.hidden_hand_success(verbose)
                draws = qq.draws
                if qq.energy >= qq.max_energy and qq.behavior == "Aggressive":
                    qq.ult(verbose)
                    ult = "Unga Bunga"
                    qq.enhanced_basic(verbose)
                    print(qq.expected_damage, qq.maximum_damage, qq.ult_expected_damage, qq.ult_max_damage)
                    #Damage_List_Update(qq.expected_damage, qq.maximum_damage, qq.ult_expected_damage, qq.ult_max_damage, "Enhanced Basic + Unga Bunga Ult", arena, qq)
                    qq_attack = "Enhanced Basic"
            else:
                qq.hidden_hand_failure(verbose)
                if qq.energy >= qq.max_energy and qq.behavior == "Emergency":
                    ult = "Emergency"
                    qq.ult(verbose)
                    atk_bonus = 0.798
                    qq.attack_percent += atk_bonus
                    qq.buffs.append({"attribute name":"attack_percent", "recipient":qq, "tickdown":qq, "value":atk_bonus, "duration":1})
                    qq.enhanced_basic(verbose)
                    #Damage_List_Update(qq.expected_damage, qq.maximum_damage, qq.ult_expected_damage, qq.ult_max_damage, "Enhanced Basic + Emergency Ult", arena, qq)
                    qq_attack = "Enhanced Basic"
                else:
                    qq.basic(verbose)
                    qq_attack = "Basic"
                    #Damage_List_Update(qq.expected_damage, qq.maximum_damage, qq.ult_expected_damage, qq.ult_max_damage, "Sad Basic", arena, qq)
             
            qq.update_buffs(verbose)
            qq.action_reset()
        
        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges


